### **Weak Lensing Testing Notebook**

Notebook for testing of weak-lensing computed profiles for galaxy clusters in DES. The following lines can be used to analyze the structure of the dataset.
```
cd /project/projectdirs/des/www/y3_cats/
h5ls -f -r --follow-symlinks Y3_mastercat_03_31_20.h5
```

#### **Initialize Notebook and Load Catalog**

Initializes the notebook and its package dependencies. Loads Y3 master catalog.

In [1]:
import numpy as np
import treecorr
import h5py
import kmeans_radec
import seaborn as sns
import matplotlib.pyplot as plt 
import astropy.units as u
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
import sys

cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

sns.set(style='ticks')

%config IPCompleter.greedy = True
%config InlineBackend.figure_format = 'retina'

def progress_bar(cur_val, final_val):
    """ 
    Function to keep track of progress during computations by displaying
    a progress bar

    Parameters:
    cur_val (int/float): current iteration/value calculation is on
    final_val (int/float): final iteration/value that calculation will take
    """

    bar_length = 20
    percent = float(cur_val) / final_val
    arrow = '-' * int(round(percent * bar_length)-1) + '>'
    spaces = ' ' * (bar_length - len(arrow))

    sys.stdout.write("\rProgress: [{0}]"
                     " {1}%".format(arrow + spaces, int(round(percent * 100))))
    sys.stdout.flush()
    
    
cat_str = '/project/projectdirs/des/www/y3_cats/Y3_mastercat_03_31_20.h5'

f = h5py.File(cat_str,'r')

In [ ]:
# red_magic_str = '/project/projectdirs/des/www/y3_cats/y3_gold_2.2.1_wide_sofcol_run_redmapper_v0.5.1_redmagic_12_3_19.h5'


# def GenerateJKRegions(ra, dec, njack, maxiter=200, tol=1.0e-5):
#     """
#     Generate k-means clusters from a set of data, using `kmeans_radec <https://github.com/esheldon/kmeans_radec>`_.
#     For roughly uniform data, it generates N-clusters of roughly equal cardinality.
#     Here, distances are computed on the surface of the unit sphere, and coordinates are given as RA/DEC.

#     Parameters
#     ----------
#     ra (float array)
#         Right ascension values for each data point.
#     dec (float array)
#         Declination values for each data point.
#     njack (int)
#         Number of k-means clusters to generate, i.e. the number of JK regions.
#     jfile (str)
#         Output file name to save the regions.
#     maxiter (int)
#         Maximum number of iterations for the k-means generation, see `kmeans_radec documentation
#         <https://github.com/esheldon/kmeans_radec>`_.
#     tol (float)
#         Tolerance level needed to be considered converged, see `kmeans_radec documentation
#         <https://github.com/esheldon/kmeans_radec>`_.

#     Returns
#     -------
#     km_centers (float)
#         Centers of selected k_means arrays

#     """

#     rd = np.zeros((len(ra), 2))
#     rd[:, 0] = ra
#     rd[:, 1] = dec

#     km = kmeans_radec.kmeans_sample(rd, njack, maxiter=maxiter, tol=tol)

#     if not km.converged:
#         raise RuntimeError("k means did not converge")

#     return km.centers
    
# ra = np.array([], dtype=float)
# dec = np.array([], dtype=float)

# with h5py.File(red_magic_str, 'r') as f:
#     ra = np.array(f['catalog']['redmagic']['combined_sample_fid']['ra']).copy()
#     dec = np.array(f['catalog']['redmagic']['combined_sample_fid']['dec']).copy()

#### **Load and Setup Galaxy and Shear Data**

Loads the positions and redshifts (bpz) of galaxies with robust shear estimates alongside shear estimates. Creates jackknife patches.

In [102]:
### Load ra, dec, redshift
print("Loading Catalog Data:")
ra_gal = np.array(f['catalog/gold/ra'])[np.array(f['index/select'])].copy() #'index/select' ensures appropriate shear measurment
dec_gal = np.array(f['catalog/gold/dec'])[np.array(f['index/select'])].copy()

z_gal = np.array(f['catalog/bpz/unsheared/zmean_sof'])[np.array(f['index/select'])].copy()
z_mc_gal = np.array(f['catalog/bpz/unsheared/zmc_sof'])[np.array(f['index/select'])].copy()

# Remove non-applicable data points identified by negative redshift 
ind_z = np.where(z_gal > 0)[0]
ra_gal = ra_gal[ind_z]
dec_gal = dec_gal[ind_z]
z_gal = z_gal[ind_z]
z_mc_gal = z_mc_gal[ind_z]

# Convert right ascension to take into account DES footprint
ra_tem = ra_gal.copy()
ra_tem[ra_tem > 250.] -= 360.
ra_dec_gal = np.vstack((ra_tem, dec_gal)).T

# Load shear data
print("Loading Shear Data:")
dgamma = 2*0.01
R = 0.5*(np.array(f['catalog/metacal/unsheared/R11'])[np.array(f['index/select'])][ind_z]+np.array(f['catalog/metacal/unsheared/R22'])[np.array(f['index/select'])][ind_z]).copy()
Rs = 0.0120708305447 #selection response

e1 = np.array(f['catalog/metacal/unsheared/e_1'])[np.array(f['index/select'])][ind_z].copy()
e2 = np.array(f['catalog/metacal/unsheared/e_2'])[np.array(f['index/select'])][ind_z].copy()

# Loading Cluster Data (Same as above):
ra_cl = np.array(f['catalog']['redmapper']['lgt20']['ra']).copy()
dec_cl = np.array(f['catalog']['redmapper']['lgt20']['dec']).copy()
z_cl = np.array(f['catalog']['redmapper']['lgt20']['z']).copy()

ind_z = np.where(z_cl > 0)[0]
ra_cl = ra_cl[ind_z]
dec_cl = dec_cl[ind_z]
z_cl = z_cl[ind_z]

ra_tem = ra_cl.copy()
ra_tem[ra_tem > 250.] -= 360.
ra_dec_cl = np.vstack((ra_tem, dec_cl)).T

print("Computing Comoving Distance:")
dist_gal = cosmo.comoving_distance(z_gal)
dist_mc_gal = cosmo.comoving_distance(z_mc_gal)
dist_cl = cosmo.comoving_distance(z_cl)

Loading Catalog Data:
Loading Shear Data:
Computing Comoving Distance:


#### **Split into 100 JK Patches**

Split data for clusters and galaxies into 100 jackknife patches.

In [5]:
# Find jackknife patch of each galaxy
N_GAL = len(ra_dec_gal)
N_CL = len(ra_dec_cl)
n_jack = 100

jk_patch_gal = np.zeros(N_GAL)
jk_patch_cl = np.zeros(N_CL)

def find_nearest(coord, centers):
    
    return kmeans_radec.find_nearest(coord, centers)[0]


def find_jk_gal(ra_dec, km):
    
    for i in range(1003):
        progress_bar(i, 1003)

        if i < 1002:
            jk_patch_gal[i*100000:(i+1)*100000] = find_nearest(ra_dec[i*100000:(i+1)*100000], km)
        else:
            jk_patch_gal[i*100000:] = find_nearest(ra_dec[i*100000:], km)

        progress_bar(i+1, 1003)
    
    return jk_patch_gal
    
jk_patch_gal_str = "/global/cscratch1/sd/samgolds/DES_WL/jk_patches_gal.npy"
jk_patch_dm_str = "/global/cscratch1/sd/samgolds/DES_WL/jk_patches_dm.npy"
center_data = np.load("km_centers.npy")
jk_patch_gal = np.load(jk_patch_gal_str)
jk_patch_cl = np.load(jk_patch_dm_str)

#### **Compute Profiles (Single Profile Example)**

Compute profiles for each jackknife patch separated in redshift bins. Below I currently just try and compute a single profile for a JK patch at a single redshift bin and will implement the looping over patches and redshifts once I have this figured out.

In [28]:
pos_jk = SkyCoord(ra=center_data.T[0]*u.deg, dec=center_data.T[1]*u.deg) # position of the jk centers
ang_max = 20.8 # Maximum angular scale to include: modify based on radius and JK patch size


z_bins = [0.2, 0.4, 0.6, 0.8] # Example bin test 

NZBIN = len(z_bins)-1
NBINS = 30

top = np.zeros((NZBIN, NBINS))
top_im = np.zeros((NZBIN, NBINS))
bottom = np.zeros((NZBIN, NBINS))
weight = np.zeros((NZBIN, NBINS))

# Set jackknife index to 0 (CONVERT TO FOR-LOOP LATER)
i = 0  

# Select galaxies around JK patch
# Select all jk patches below angular scale
jk_dist = pos_jk[i].separation(pos_jk).degree # the distance to each JK center from each JK center
jk_neigh_ind = np.where(jk_dist < ang_max)[0]

# Select galaxy/shear JK data
ind_jk_gal = np.in1d(jk_patch_gal, jk_neigh_ind) ### selected galaxies that have the above JK indices

ra_gal_jk = ra_gal[ind_jk_gal]
dec_gal_jk = dec_gal[ind_jk_gal]
z_gal_jk = z_gal[ind_jk_gal]
z_mc_gal_jk = z_mc_gal[ind_jk_gal]
dist_gal_jk = dist_gal[ind_jk_gal]
dist_mc_gal_jk = dist_mc_gal[ind_jk_gal]

R_jk = R[ind_jk_gal]
e1_jk = e1[ind_jk_gal]
e2_jk = e2[ind_jk_gal]

# Select cluster JK data
ind_jk_cl = np.in1d(jk_patch_cl, jk_neigh_ind) ### selected clustrs that have the above JK indices

ra_cl_jk = ra_cl[ind_jk_cl]
dec_cl_jk = dec_cl[ind_jk_cl]
z_cl_jk= z_cl[ind_jk_cl]

# Select clusters in this redshift bin (CONVERT TO FOR-LOOP LATER)
j = 0 
z_min= z_bins[j]
z_max = z_bins[j+1]

z_ind_cl = np.where((z_cl_jk > z_min) & (z_cl_jk < z_max))[0]

# Initialize cluster catalog
cat_cl = treecorr.Catalog(ra=ra_cl_jk[z_ind_cl], dec=dec_cl_jk[z_ind_cl], 
                          a_units='deg', dec_units='deg')
z_cl_zbin = np.mean(z_cl_jk[z_ind_cl])

""" Removed conversion factor (10**6*h) below as result is already in Mpc"""
dist_cl_zbin = cosmo.comoving_distance(z_cl_zbin).value  

# Compute weights
"""I'm not sure I have everything in the correct units for this"""
wt_gal = (1./(1.663*10**12))*(1+z_cl_zbin)*dist_cl_zbin*(1.-dist_cl_zbin/dist_gal_jk.value)
sci = (1./(1.663*10**12))*(1.+z_cl_zbin)*dist_cl_zbin*(1.-dist_cl_zbin/dist_mc_gal_jk.value)
sci[sci<0.] = 0. ###### important


# Select galaxies in front of cluster (with cushion)
gal_sel = (z_gal_jk >= z_cl_zbin+0.1) 

# Construct galaxy catalogs
"""Is the below equality true in WL regime? If not, how can I find gamma?"""
gamma1 = e1_jk[gal_sel]
gamma2 = e2_jk[gal_sel]

cat_gal = treecorr.Catalog(g1=gamma1, g2=gamma2, ra=ra_gal_jk[gal_sel], dec=dec_gal_jk[gal_sel],
                           w=wt_gal[gal_sel], ra_units='deg', dec_units='deg')

cat_gal2 = treecorr.Catalog(g1=gamma1, g2=gamma2, ra=ra_gal_jk[gal_sel], dec=dec_gal_jk[gal_sel], 
                            w=(wt_gal*sci*(R_jk+Rs))[gal_sel], ra_units='deg', dec_units='deg')


"""Is this correct? I am assuming Rmin and Rmax are in Mpc so I just put in some"""
Rmin = 0.1
Rmax = 2

th_min = np.arctan(Rmin*10**6/dist_cl_zbin)*(180./np.pi)*60. ### minimum angular distance in arcmin
th_max = np.arctan(Rmax*10**6/dist_cl_zbin)*(180./np.pi)*60.

ng = treecorr.NGCorrelation(nbins=NBINS, min_sep=th_min, max_sep=th_max,
                            sep_units='arcmin', verbose=2, bin_slop=0.02) 
ng2 = treecorr.NGCorrelation(nbins=NBINS, min_sep=th_min, max_sep=th_max,
                             sep_units='arcmin', verbose=2,bin_slop=0.02)

ng.process(cat_cl, cat_gal) 
ng2.process(cat_cl, cat_gal2)

top[j] = ng.xi*ng.weight #### remove the denominator part so that we are only left with the numerator
top_im[j] = ng.xi_im*ng.weight
bottom[j] = ng2.weight #### we selected the denominator part with the different weight (R*sci*wt)
weight[j] = ng.weight #### for calculating boost factor in the future

### do it for every JK patch and combine

top_total = sum(top)
bottom_total = sum(bottom)

## and then
top_total/bottom_total

### do this for 100 JK realization and calculate JK cov.   